In [ ]:
pip install python-tsp

In [58]:
#Importando Bibliotecas
import pandas as pd
import urllib.request, json
import requests
import numpy as np
from python_tsp.exact import solve_tsp_dynamic_programming

In [56]:
#Definindo CEP's de entrada para o modelo
data = pd.DataFrame(['01001000','13040709','13484350','13083710','13970390','13971260',
                     '13087506','13035000','13073003','03065010','04142091','13480357',
                     '13098422','13331530','13400330','13065030','13348290','86055660',
                     '13348884'], columns = ["CEP"])
                     
data

,CEP
0,01001000
1,13040709
2,13484350
3,13083710
4,13970390
5,13971260
6,13087506
7,13035000
8,13073003
9,03065010


In [57]:
#Coletando Dados de Endereço
rua = []
bairro = []
cidade = []
estado = []

for i in data["CEP"]:
    with urllib.request.urlopen(f"https://viacep.com.br/ws/{i}/json/") as url:
        address = json.loads(url.read().decode())
    rua.append(   address['logradouro'])
    bairro.append(address['bairro'])
    cidade.append(address['localidade'])
    estado.append(address['uf'])

data["Rua"]    = rua
data["Bairro"] = bairro
data["Cidade"] = cidade
data["Estado"] = estado

data

,CEP,Rua,Bairro,Cidade,Estado
0,01001000,Praça da Sé,Sé,São Paulo,SP
1,13040709,Rua Ezequiel Foga,Loteamento Parque São Martinho,Campinas,SP
2,13484350,Rua Pedro Zaccaria,Jardim Santa Luiza,Limeira,SP
3,13083710,Rua Doutor Ruberlei Boareto da Silva,Cidade Universitária,Campinas,SP
4,13970390,Rua da Penha,Chácara Freitas,Itapira,SP
5,13971260,Rua David Faraco,Jardim Galego,Itapira,SP
6,13087506,Rua Clóvis Teixeira,Mansões Santo Antônio,Campinas,SP
7,13035000,Rua Francisco de Assis Pupo,Vila Industrial,Campinas,SP
8,13073003,Rua Dom Pedro I,Jardim Brasil,Campinas,SP
9,03065010,Rua Potiguares,Tatuapé,São Paulo,SP


In [6]:
#Determinação de matrizes de Distâncias e Tempo nulas
l = len(data["CEP"])
dist_matrix = np.zeros((l,l), dtype = np.float64)
time_matrix = np.zeros((l,l), dtype = np.float64)

#Atribuição de Valores às Matrizes de Distância e Tempo
for i in range(l):
    for j in range(i+1,l):
        api_key  = Your_Key
        
        rota = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={data['Rua'][i]}-{data['Bairro'][i]}-{data['Cidade'][i]}-{data['Estado'][i]}&destinations={data['Rua'][j]}-{data['Bairro'][j]}-{data['Cidade'][j]}-{data['Estado'][j]}&key={api_key}" #URL de coleta de dados
        response = requests.get(url = rota)
        result = response.json()
        dist_matrix[i][j] = (float(result["rows"][0]['elements'][0]['distance']['value'])) #Criação da Matriz de distâncias triângular superior
        dist_matrix[j][i] = dist_matrix[i][j] #Criação da simetria da matriz
        time_matrix[i][j] = (float(result["rows"][0]['elements'][0]['duration']['value'])) #Criação da Matriz de distâncias triângular superior
        time_matrix[j][i] = time_matrix[i][j] #Criação da simetria da matriz

dist_matrix = pd.DataFrame(dist_matrix)
time_matrix = pd.DataFrame(time_matrix)

In [7]:
dist_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.0,87677.0,160806.0,107135.0,162064.0,164832.0,103890.0,93170.0,96389.0,8348.0,9538.0,146051.0,105595.0,108486.0,157065.0,99007.0,108616.0,565191.0,109777.0
1,87677.0,0.0,62963.0,25758.0,83977.0,86744.0,25802.0,7241.0,10460.0,91753.0,98425.0,58366.0,27508.0,33479.0,74352.0,13083.0,33609.0,551418.0,34770.0
2,160806.0,62963.0,0.0,58279.0,72459.0,75227.0,60542.0,58010.0,59151.0,164838.0,171510.0,4024.0,63655.0,96294.0,36263.0,51749.0,96424.0,512858.0,97584.0
3,107135.0,25758.0,58279.0,0.0,67637.0,70404.0,5237.0,12321.0,8259.0,111140.0,117811.0,53191.0,11168.0,47948.0,69572.0,15230.0,48078.0,565887.0,49238.0
4,162064.0,83977.0,72459.0,67637.0,0.0,4222.0,63669.0,72264.0,67892.0,166855.0,173527.0,70452.0,58342.0,110341.0,106905.0,77623.0,110471.0,578997.0,111631.0
5,164832.0,86744.0,75227.0,70404.0,4222.0,0.0,66830.0,75425.0,71054.0,170016.0,176688.0,73613.0,61503.0,113502.0,110066.0,80784.0,113632.0,582158.0,114793.0
6,103890.0,25802.0,60542.0,5237.0,63669.0,66830.0,0.0,9690.0,5564.0,110130.0,116802.0,54931.0,10158.0,49688.0,71313.0,16970.0,49818.0,567627.0,50979.0
7,93170.0,7241.0,58010.0,12321.0,72264.0,75425.0,9690.0,0.0,4498.0,96843.0,103514.0,53673.0,18611.0,32607.0,70074.0,8805.0,32737.0,550546.0,33898.0
8,96389.0,10460.0,59151.0,8259.0,67892.0,71054.0,5564.0,4498.0,0.0,99685.0,106357.0,52903.0,14565.0,35449.0,70126.0,8857.0,35580.0,553389.0,36740.0
9,8348.0,91753.0,164838.0,111140.0,166855.0,170016.0,110130.0,96843.0,99685.0,0.0,13508.0,148938.0,108482.0,111373.0,159952.0,101894.0,111503.0,568078.0,112664.0


In [8]:
time_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.0,4302.0,6925.0,5333.0,7532.0,7955.0,5096.0,4607.0,5156.0,1276.0,1257.0,6713.0,5088.0,5720.0,7165.0,4891.0,5260.0,23721.0,5474.0
1,4302.0,0.0,3294.0,1466.0,3855.0,4278.0,1419.0,580.0,1129.0,4472.0,5313.0,2857.0,1411.0,2384.0,3620.0,872.0,1923.0,22633.0,2138.0
2,6925.0,3294.0,0.0,3190.0,3524.0,3948.0,3195.0,3189.0,3196.0,7043.0,7883.0,650.0,3274.0,4626.0,2158.0,2838.0,4165.0,22304.0,4380.0
3,5333.0,1466.0,3190.0,0.0,3268.0,3691.0,627.0,1270.0,802.0,5392.0,6233.0,2795.0,824.0,3039.0,3745.0,1112.0,2579.0,23289.0,2794.0
4,7532.0,3855.0,3524.0,3268.0,0.0,644.0,2872.0,3789.0,3260.0,7560.0,8400.0,3333.0,2559.0,5417.0,5588.0,3491.0,4957.0,24777.0,5172.0
5,7955.0,4278.0,3948.0,3691.0,644.0,0.0,3377.0,4294.0,3765.0,8065.0,8905.0,3839.0,3065.0,5923.0,6093.0,3996.0,5462.0,25283.0,5677.0
6,5096.0,1419.0,3195.0,627.0,2872.0,3377.0,0.0,1290.0,734.0,5319.0,6159.0,2807.0,750.0,3051.0,3757.0,1124.0,2591.0,23301.0,2806.0
7,4607.0,580.0,3189.0,1270.0,3789.0,4294.0,1290.0,0.0,699.0,4740.0,5580.0,2760.0,1458.0,2400.0,3581.0,834.0,1940.0,22650.0,2155.0
8,5156.0,1129.0,3196.0,802.0,3260.0,3765.0,734.0,699.0,0.0,5238.0,6078.0,2883.0,1003.0,2899.0,3765.0,1017.0,2438.0,23148.0,2653.0
9,1276.0,4472.0,7043.0,5392.0,7560.0,8065.0,5319.0,4740.0,5238.0,0.0,1823.0,6634.0,5009.0,5641.0,7086.0,4812.0,5180.0,23642.0,5395.0


In [45]:
#Solver TSP por Distâncias

dist, distance1 = solve_tsp_dynamic_programming(np.array(dist_matrix))
optimizing_distance = []
for i in dist:
    optimizing_distance.append(f"{data['Rua'][i]} - {data['Bairro'][i]}, {data['Cidade'][i]} - {data['Estado'][i]}")

#Solver TSP por Tempo

time, distance2 = solve_tsp_dynamic_programming(np.array(time_matrix))
optimizing_time = []
for i in time:
    optimizing_time.append(f"{data['Rua'][i]} - {data['Bairro'][i]}, {data['Cidade'][i]} - {data['Estado'][i]}")

dist_order = pd.DataFrame(optimizing_distance, columns = ["Ordem de entrega por distância"])
time_order = pd.DataFrame(optimizing_time    , columns = ["Ordem de entrega por tempo"])

In [53]:
dist_order

,Ordem de entrega por distância
0,"Praça da Sé - Sé, São Paulo - SP"
1,Rua Ezequiel Foga - Loteamento Parque São Mart...
2,"Rua Francisco de Assis Pupo - Vila Industrial,..."
3,"Rua Matheus Maccari - Parque Via Norte, Campin..."
4,"Rua Dom Pedro I - Jardim Brasil, Campinas - SP"
5,"Rua Clóvis Teixeira - Mansões Santo Antônio, C..."
6,Rua Doutor Ruberlei Boareto da Silva - Cidade ...
7,Rua Ângelo Sellin - Jardim Myrian Moreira da C...
8,"Rua da Penha - Chácara Freitas, Itapira - SP"
9,"Rua David Faraco - Jardim Galego, Itapira - SP"


In [54]:
time_order

,Ordem de entrega por tempo
0,"Praça da Sé - Sé, São Paulo - SP"
1,"Rua Potiguares - Tatuapé, São Paulo - SP"
2,"Rua Viale Itália - Jardim Vila Paradiso, Indai..."
3,Rua Lucimeire de Fátima Coleone - Jardim Regin...
4,Rua Antônio Angelino Rossi - Jardim Morada do ...
5,"Rua Matheus Maccari - Parque Via Norte, Campin..."
6,Rua Ezequiel Foga - Loteamento Parque São Mart...
7,"Rua Francisco de Assis Pupo - Vila Industrial,..."
8,"Rua Dom Pedro I - Jardim Brasil, Campinas - SP"
9,Rua Doutor Ruberlei Boareto da Silva - Cidade ...
